# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [8]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)



# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (D - A) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [9]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [10]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [11]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [12]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [13]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [14]:
def Jacobi_iter(A, b, max_iter):
    diag_1 = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    D = np.diag(diag_1)
    invD = np.diag(1./diag_1)
    BB = invD @ B 
    c = invD @ b
    
    x = np.ones(A.shape[0])
    for _ in range(max_iter):
        x = BB @ x + c
    
    return x

In [15]:
x2 = Jacobi_iter(A, b, 50)

A @ x2 - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

In [16]:
A[np.diag_indices_from(A)] /= 20 #make diagonal elements of A smaller

diag_2 = np.diag(A)
B2 = -A.copy()
np.fill_diagonal(B2, 0)
D2 = np.diag(diag_2)
invD2 = np.diag(1./diag_2)
BB2 = invD2 @ B2

np.linalg.norm(BB2)

7.287232396603068

The condition $$ \|B\| < 1 $$ is equivalent to A being row diagonally dominant, and is a sufficient condition for convergence. So, Jacobi method won't converge.

In [17]:
x3 = Jacobi_iter(A, b, 100)

A @ x3 - b

array([3.82809888e+78, 3.34232163e+78, 3.16535310e+78, 3.94548471e+78,
       2.07976084e+78, 3.34791701e+78, 2.11693907e+78, 2.58900771e+78,
       3.49906469e+78, 3.04248192e+78])

These NaNs appear due to overflow (machine infinity)

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [18]:
rndm = np.random.RandomState(1234)

n = 10
#A = rndm.uniform(size=(n, n))
A = rndm.randint(0, 100000, size = (n,n))
#b = rndm.uniform(size=n)
b = rndm.randint(0, 100000, size = n)
A = np.tril(A)  #creating a triangilar matrix for Seidel's method to converge


In [19]:
def Seidel_iter(A, b, max_iter):
    B = A.copy()
    diag_1 = np.diag(B)
    D = np.diag(diag_1)
    L = np.tril(B, -1) #lower triangular matrix with zeros on the main diagonal
    U = np.triu(B, 1)
    DpL_inv = np.linalg.inv(D + L)
    BB = DpL_inv @ U
    c = DpL_inv @ b
    x = np.ones(B.shape[0])
    for _ in range(max_iter):
        x = -BB @ x + c
    return x, BB #return BB only to check conditions of convergence later on 

In [20]:
x, BB = Seidel_iter(A, b, 100)

A @ x - b

array([ 2.18278728e-11,  1.45519152e-11, -5.82076609e-11, -5.82076609e-11,
       -1.74622983e-10,  0.00000000e+00,  3.49245965e-10, -4.65661287e-10,
       -1.16415322e-10, -2.32830644e-10])

In [21]:
print(np.linalg.norm(BB, ord = np.inf))
print(np.linalg.norm(BB, ord = 1))

0.0
0.0


Both inf-norm and 1-norm are zeroes because our BB matrix values are so small that they underflow to zero even if matrix A has values like 100000

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [139]:
def min_res(A, b, max_iter):
    A1 = A.copy()
    x = np.ones(A1.shape[0])
    for i in range(max_iter):
        res_n = A1 @ x - b
        norm = np.linalg.norm(A1 @ res_n)
        if norm == 0:
            break
        tau_next = (res_n @ A1 @ res_n) / (norm ** 2) 
        #print(tau_next)
        x = x - (tau_next * res_n) 
    return x, i

In [144]:
n = 10
rndm = np.random.RandomState(123)
A = rndm.uniform(size=(n, n))
b = rndm.uniform(size = n) 


x, it = min_res(A, b, 100)

print("Iterations:" + str(it) + ", root:" + str(x))
A @ x - b

Iterations:99, root:[ 1.79661141e-02  2.55766053e-01  2.85574376e-02  1.57003318e-01
  6.10438645e-02  8.66272382e-02 -3.45867931e-05  1.81619055e-01
  9.10831437e-02  6.63299471e-02]


array([-0.05966419, -0.24831948,  0.42149358,  0.40607066,  0.13451906,
       -0.06066065, -0.40232821, -0.05220382, -0.40338393,  0.22558069])

My method behaves badly when i simply fill matrix A with random integers

In [145]:
A = np.tril(A)

x, it = min_res(A, b, 1000)
A @ x - b

array([ 0.02236172,  0.01696515,  0.07397027, -0.16209987, -0.07984139,
       -0.45673867, -0.24160662,  0.10442583,  0.03052831,  0.60363515])

It looks better when we use Seidel method's condition - a triangular matrix, but it's still bad


In [173]:
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

print("Iterations:" + str(it) + ", root:" + str(x))
x, it = min_res(A, b, 100)
A @ x - b

Iterations:23, root:[0.00245413 0.04541407 0.00331114 0.00047118 0.0571877  0.03483625
 0.007839   0.03672017 0.01350848 0.00216622]


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -1.11022302e-16,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

It works quite well with diagonally dominant matrix A. The method rarely uses all iterations since the norm of the residual quickly overflows to zero (or underflows to zero, idk how to say it)

In the first example the coefficient tau rapidly overflows to zero. In the second example it takes on a limited set of values and changes its sign so actually it doesn't do anything and the method treads water. In the third example it changes slightly at each step, starting at 0.048949 and stopping at 0.062082 on the 20-th iteration (after that it remains constant).

In [177]:
xx = np.linalg.solve(A, b) 
print("Our solution: " + str(x) + "\nGround truth solution:" + str(xx))
print(" \nThe difference: ", str(abs(np.linalg.norm(x) - np.linalg.norm(xx))))

Our solution: [ 0.050477   -0.00539249  0.03884936  0.03354195  0.02830406  0.05061043
  0.03307114  0.04814944  0.04373544  0.04540488]
Ground truth solution:[ 0.050477   -0.00539249  0.03884936  0.03354195  0.02830406  0.05061043
  0.03307114  0.04814944  0.04373544  0.04540488]
 
The difference:  2.7755575615628914e-17
